Neste primeiro bloco é realizada a importação das bibliotecas utilizadas no código.
- **matplotlib**: Utilizada para impressão dos gráficos dos resultados dos modelos
- **pandas**: Utilizada para manipulação do conjunto de dados
- **numpy**: Utilizada para operação matemáticas em cima do conjunto de dados
- **keras**: Utilizada para criação da rede neural
- **time**: Utilizada para marcar o tempo de execução do código

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras

from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.utils import plot_model
from keras import optimizers
from keras.utils import to_categorical

import time

from sklearn import preprocessing

Todas as configurações referentes a rede neural estão no bloco abaixo. Aqui é possível escolher qual conjunto de dados será utilizado, se o código deverá ou não remover as colunas correlacionadas, como será a divisão de dados entre treino e teste, a função de ativação utilizada e a taxa de aprendizado.

In [ ]:
# Flag para remoção das colunas correlacionadas.
CLEAN_DATASET = True

# Flag para decisão de qual versão do dataset usar
DATASET_COMPLETO = False

# Caminho para o dataset
# Endereço para download: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
# Backup: http://web.archive.org/web/*/http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
if DATASET_COMPLETO:
    DATASET_PATH = "D:/UNIFEI/TCC/kdd_dl/dataset/kddcup.data.corrected"   
    # Treino 98%, Validação 1%, Teste 1%
    # https://stackoverflow.com/a/13613316
    TRAIN_PERCENTAGE = 0.98
    VALIDATION_PERCENTAGE = 0.01
    TEST_PERCENTAGE = 0.01
else:
    DATASET_PATH = "D:/UNIFEI/TCC/kdd_dl/dataset/kddcup.data_10_percent_corrected"
    # Treino 80%, Validação 10%, Teste 10%
    TRAIN_PERCENTAGE = 0.8
    VALIDATION_PERCENTAGE = 0.1
    TEST_PERCENTAGE = 0.1
    
## Threshold de correlação para exclusão das colunas do dataset
CORRCOEF_THRESHOLD = 0.5

## Ignora os warnings de divisão por NaN
np.seterr(divide='ignore', invalid='ignore')


EPOCHS = 100 # Quantas vezes os dados serão corridos completamente
BATCH_SIZE = 1024 # Quantas linhas do dataset serão lidas por vez
SHUFFLE = True # Mistura os dados antes do treino
VERBOSE = 1 # 0 - Desativa os logs durante o treino, 1 - Imprime os logs durante o treino
LEARNING_RATE = 0.01 # Taxa de aprendizado
ACTIVATION_FN = "relu" # Função de ativação

O bloco abaixo realiza a leitura do conjunto de dados e cria um *dataframe* do *pandas* para manipulação mais fácil dos dados.

In [ ]:
# Nome das colunas do dataset
column_names = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes",
    "land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in",
    "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations",
    "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login", "is_guest_login",
    "count", "srv_count", "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
    "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count",
    "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate",
    "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "attack",
]

#Leitura do dataset
dataset = pd.read_csv(
    DATASET_PATH,
    header = None,
    names = column_names)

Como o conjunto de dados possui valores léxicos, é necessário trocá-los por valores numéricos.

In [ ]:
#Pega os valores  únicos das colunas que tem string
protocol_type_values = dataset.protocol_type.unique()
service_values = dataset.service.unique()
flag_values = dataset.flag.unique()
attack_values = dataset.attack.unique()

#Transforma as strings em números
protocol_type_dict = dict(zip(protocol_type_values, range(len(protocol_type_values))))
service_dict = dict(zip(service_values, range(len(service_values))))
flag_dict = dict(zip(flag_values, range(len(flag_values))))
attack_dict = dict(zip(attack_values, range(len(attack_values))))

In [ ]:
#Substitui os valores no dataset que são string por números
# dataset = dataset.replace(
#     {"protocol_type": protocol_type_dict,
#      "service": service_dict,
#      "flag": flag_dict,
#      "attack": attack_dict}
# )
dataset = dataset.replace({"attack": attack_dict})

# Pego as colunas numéricas
numeric_cols = dataset._get_numeric_data().columns
print(numeric_cols)

# Pego as colunas categoricas
categorical_cols = list(set(dataset.columns)-set(numeric_cols))
print(categorical_cols)

#Normalizo só as numéricas
names_to_normalize = list(dataset.drop(categorical_cols, axis=1).columns)
print(names_to_normalize)


min_max_scaler = preprocessing.MinMaxScaler()
data_scaled = dataset.copy(deep=True)
data_scaled[names_to_normalize] = min_max_scaler.fit_transform(
    dataset[names_to_normalize])

print(data_scaled.head())

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
def apply_dummies(df, feature):
    get_dummies = pd.get_dummies(df[feature])
    for x in get_dummies.columns:
        dummy_name = f"{feature}-{x}"
        df[dummy_name] = get_dummies[x]
    df.drop(feature, axis=1, inplace=True)
    return None

for feature in categorical_cols:
    if feature not in ['attack', 'target_type']:
        apply_dummies(data_scaled, feature)

print("#\n#")
print(data_scaled.head())
        
dataset = data_scaled

Limpeza do dataset

In [ ]:
#Apaga as colunas que tem correlação entre elas que seja maior que CORRCOEF_THRESHOLD

#Excluindo só as colunas numéricas

if CLEAN_DATASET:
    excluded = list()
    
    for index_1 in range(len(numeric_cols)-1):
        for index_2 in range(len(numeric_cols)-1):
            if numeric_cols[index_1] not in excluded and numeric_cols[index_1] in numeric_cols and numeric_cols[index_2] not in excluded and index_1 != index_2:
                corrcoef = np.corrcoef(dataset[numeric_cols[index_1]], dataset[numeric_cols[index_2]])

                if abs(corrcoef[0][1]) > CORRCOEF_THRESHOLD:
                    excluded.append(numeric_cols[index_2])
                
    dataset = dataset[dataset.columns.difference(excluded)]

Limpeza do dataset

In [ ]:
#Separa os dados entre treino e teste
train, test = np.split(dataset.sample(frac=1), [int(TRAIN_PERCENTAGE * len(dataset))])

train_samples = train.drop(['attack'], axis=1)
train_labels = to_categorical(train[['attack']].to_numpy(), num_classes=len(attack_values))

test_samples = test.drop(['attack'], axis=1)
test_labels = to_categorical(test[['attack']].to_numpy(), num_classes=len(attack_values))

Modelo da rede neural

In [ ]:
input_shape = (train_samples.shape[1],)

# Dense(15, activation=ACTIVATION_FN),
# Criação do modelo
model = Sequential([
    Dense(1, input_shape=input_shape, activation=ACTIVATION_FN),
    Dense(1, activation=ACTIVATION_FN),
    Dense(len(attack_values), activation='softmax'),
])

In [ ]:
sgd = optimizers.SGD(lr=LEARNING_RATE)

model.compile(
    optimizer=sgd,
    loss='mean_squared_error', 
    metrics=['accuracy'],
)

# categorical_crossentropy
# mean_squared_error

Treino da rede neural

In [ ]:
start_time = time.time()

# Treinamento do modelo
history = model.fit(
    x=train_samples, 
    y=train_labels, 
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS,
    shuffle=SHUFFLE, #Mistura os dados
    verbose=VERBOSE,
    validation_split=VALIDATION_PERCENTAGE #Porcentagem dos dados de treino que serão usadas para validação
)

end_time = time.time()

Teste da rede neural

In [ ]:
# Teste do modelo
h2 = model.evaluate(
    x=test_samples,
    y=test_labels,
    verbose=VERBOSE,
)
# plot_model(model, to_file='model.png' ,show_shapes=True, show_layer_names=True, expand_nested=True, dpi=200)

Impressão dos resultados

In [ ]:
# plot_model(model, to_file='model.png')

# Plot training & validation accuracy values
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

print(model.metrics_names)
print(h2)

print({
"CLEAN_DATASET":CLEAN_DATASET,
"DATASET_COMPLETO":DATASET_COMPLETO,
"CORRCOEF_THRESHOLD":CORRCOEF_THRESHOLD,
"BATCH_SIZE":BATCH_SIZE,
"EPOCHS":EPOCHS,
"SHUFFLE":SHUFFLE,
"VERBOSE":VERBOSE,
"VALIDATION_SPLIT":VALIDATION_PERCENTAGE,
"LEARNING_RATE":LEARNING_RATE,
"ACTIVATION_FN":ACTIVATION_FN,
"TEMPO": end_time-start_time})

In [ ]:
# np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
# from keras import backend as K

# with a Sequential model
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[3].output])
# print(get_3rd_layer_output)

# print(model.layers[0].input)
# print(model.layers[0].output)

# print(model.layers[1].input)
# print(model.layers[1].output)

# print(model.layers[2].input)
# print(model.layers[2].output)

# print(model.layers[3].input)
# print(model.layers[3].output)

# from keras import backend as K

# input1 = model.input # input placeholder

# output1 = [layer.output for layer in model.layers]# all layer outputs

# fun = K.function([input1, K.learning_phase()],output1)# evaluation function

# # Testing

# t = np.random.random((41,))[np.newaxis,...]

# t = np.array([[0,1,19,9,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00]])

# layer_outputs = fun([t, 1])



# print(layer_outputs[0]) #printing the outputs of layers
# print(layer_outputs[1]) #printing the outputs of layers
# print(layer_outputs[2]) #printing the outputs of layers
# print(layer_outputs[3]) #printing the outputs of layers